# 2. Cartesian Offsets

As a first example, let's play around with cartesian offsets on structured grids. In the following we define a laplacian in 2 dimensions and a centered difference in the i-direction in 2 dimensions. We then combine these two operators ...

In [43]:
import numpy as np
import matplotlib.pyplot as plt

import gt4py.next as gtx
from gt4py.next import Dims

from helpers import *

In [44]:
@gtx.field_operator
def laplace(in_field: gtx.Field[[I, J], "float"]) -> gtx.Field[[I, J], "float"]:
    return (
        -4.0 * in_field
        + in_field(I + 1, J + 1)
        + in_field(I + 1, J - 1)
        + in_field(I - 1, J + 1)
        + in_field(I - 1, J - 1)
    )

In [45]:
@gtx.field_operator
def centered_difference_i(in_field: gtx.Field[[I, J], "float"]) -> gtx.Field[[I, J], "float"]:
    return in_field(I + 1) - in_field(I - 1)


In [46]:
@gtx.field_operator
def combined(in_field: gtx.Field[[I, J], "float"]) -> gtx.Field[[I, J], "float"]:
    return centered_difference_i(laplace(in_field))

In [54]:
@gtx.field_operator
def combined_naive(in_field: gtx.Field[[I, J], "float"]) -> gtx.Field[[I, J], "float"]:
    return (
        -4.0 * in_field(I + 1)
        + in_field(I + 2, J + 1)
        + in_field(I + 2, J - 1)
        + in_field(       J + 1)
        + in_field(       J - 1)
        + 4.0 * in_field(I - 1)
        - in_field(       J + 1)
        - in_field(       J - 1)
        - in_field(I - 2, J + 1)
        - in_field(I - 2, J - 1)
    )

In [65]:
def test_computation():
    backend = None
    # backend = gtfn_cpu
    # backend = gtfn_gpu
    
    structured_domain = gtx.domain({I: 10, J: 10})

    structured_result_domain = gtx.domain({I: (2, 8) , J: (1, 9)})

    input = random_field(structured_domain, allocator=backend)
 

    combined_reference = gtx.zeros(structured_result_domain, allocator=backend)

    combined(
        input,
        out=combined_reference,
        offset_provider={Ioff.value: I, Joff.value: J},
    )

    combined_naive = gtx.zeros(structured_result_domain, allocator=backend)

    combined(
        input,
        out=combined_naive,
        offset_provider={Ioff.value: I, Joff.value: J},
    )

    print(combined_reference.asnumpy())
    print(combined_naive.asnumpy())

    assert np.allclose(combined_reference.asnumpy(), combined_naive.asnumpy())

In [66]:
test_computation()

[[ 1.26041545  0.04257704 -0.4575251   2.69138062  1.86050509  3.57258153
   1.3499693  -5.02432625]
 [ 6.95512944  3.76304299  2.40852921 -2.29538083  0.04519938  1.4688093
   3.14548377  4.19020435]
 [ 0.38783363 -2.98048825 -2.52778711 -6.03286952 -5.50988761 -6.08677145
  -4.59747401  6.37860238]
 [-2.08037822  1.61458121 -2.50692437  7.26080828  3.8713706   5.62213071
   1.30764551 -2.90606554]
 [-6.75171209  0.35810108 -0.12999619  2.98429019 -3.19311727 -0.04953506
   0.08247604 -1.40756187]
 [ 3.68719054  0.34383162 -1.03542281 -3.66106553  1.65237174 -2.99180777
  -2.96578888  0.53349804]]
[[ 1.26041545  0.04257704 -0.4575251   2.69138062  1.86050509  3.57258153
   1.3499693  -5.02432625]
 [ 6.95512944  3.76304299  2.40852921 -2.29538083  0.04519938  1.4688093
   3.14548377  4.19020435]
 [ 0.38783363 -2.98048825 -2.52778711 -6.03286952 -5.50988761 -6.08677145
  -4.59747401  6.37860238]
 [-2.08037822  1.61458121 -2.50692437  7.26080828  3.8713706   5.62213071
   1.30764551 -2.9